In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error

In [67]:
installs = pd.read_csv('data/installsConFeatures.csv', low_memory=False, parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,Unnamed: 0,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,top10apps,top50marcas,top10marcas,top10marcasSinNan,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng
created,,,,,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,94738,70,1,4432995619177048534,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
2019-04-18 00:00:01.851,94737,70,1,5904733559638204455,False,False,6287817205707153877,0,208,1095,...,0,0,0,0,0,0,0,1,1,9
2019-04-18 00:00:05.152,111877,65,1,896373747754111825,False,True,6287817205707153877,6,102,2,...,3,6,6,6,2,2,1,0,0,8
2019-04-18 00:00:05.589,76146,27,0,3399210824535017892,False,False,6287817205707153877,0,190,2,...,0,0,0,0,21,14,6,1,1,2
2019-04-18 00:00:06.795,435184,339,0,1541425881979513687,False,False,6287817205707153877,0,190,2,...,0,0,0,0,68,46,10,1,1,6


In [68]:
installs.drop(['Unnamed: 0'],axis = 1,inplace=True)

In [9]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 481511 entries, 2019-04-18 00:00:01.560000 to 2019-04-26 23:59:58.788000
Data columns (total 29 columns):
application_id        481511 non-null int64
ref_type              481511 non-null int64
ref_hash              481511 non-null int64
attributed            481511 non-null bool
implicit              481511 non-null bool
device_countrycode    481511 non-null int64
device_brand          481511 non-null int64
device_model          481511 non-null int64
session_user_agent    481511 non-null int64
user_agent            481511 non-null int64
kind                  481511 non-null int64
wifi                  481511 non-null bool
ip_address            481511 non-null int64
device_language       481511 non-null int64
TopApp/Otro           481511 non-null int64
dia                   481511 non-null int64
hora                  481511 non-null int64
minuto                481511 non-null int64
diaDeLaSemana         481511 non-null int64
top10app

In [25]:
installs.drop(['ref_type','session_user_agent','wifi','minuto','top50marcas','top10marcas','top10marcasSinNan','agentes','top50agentes','top10agentes','top50agentes','kindReducido','top10tipo','top10leng'],axis=1,inplace=True)

In [69]:
info1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info1['created'] = info1['created'] - pd.Timestamp('2019-04-21 00:00:00')
info1['created'] = info1['created'].transform(lambda x: x.total_seconds())
info1 = info1.reset_index()
info1 = info1.merge(installs['2019-04-18':'2019-04-20'].reset_index(),on="ref_hash", how="right")
info1 = info1.rename(columns={'created_x':'label','created_y':'created'})

label1 = info1['label'].to_frame()
train1 = info1.drop('label', axis=1)

In [70]:
info2 = installs['2019-04-22':'2019-04-24'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info2['created'] = info2['created'] - pd.Timestamp('2019-04-22 00:00:00')
info2['created'] = info2['created'].transform(lambda x: x.total_seconds())
info2 = info2.reset_index()
info2 = info2.merge(installs['2019-04-19':'2019-04-21'].reset_index(),on="ref_hash", how="right")
info2 = info2.rename(columns={'created_x':'label','created_y':'created'})

label2 = info2['label'].to_frame()
train2 = info2.drop('label', axis=1)

In [71]:
info3 = installs['2019-04-23':'2019-04-25'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info3['created'] = info3['created'] - pd.Timestamp('2019-04-23 00:00:00')
info3['created'] = info3['created'].transform(lambda x: x.total_seconds())
info3 = info3.reset_index()
info3 = info3.merge(installs['2019-04-20':'2019-04-22'].reset_index(),on="ref_hash", how="right")
info3 = info3.rename(columns={'created_x':'label','created_y':'created'})

label3 = info3['label'].to_frame()
train3 = info3.drop('label', axis=1)

In [72]:
info4 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info4['created'] = info4['created'] - pd.Timestamp('2019-04-24 00:00:00')
info4['created'] = info4['created'].transform(lambda x: x.total_seconds())
info4 = info4.reset_index()
info4 = info4.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="right")
info4 = info4.rename(columns={'created_x':'label','created_y':'created'})

label4 = info4['label'].to_frame()
train4 = info4.drop('label', axis=1)

In [73]:
ventana_ultima = installs['2019-04-24':'2019-04-26'].reset_index()

In [74]:
train1 = train1.dropna()
train2 = train2.dropna()
train3 = train3.dropna()
train4 = train4.dropna()
ventana_ultima = ventana_ultima.dropna()
label1 = label1.fillna(3*24*60*60)
label2 = label2.fillna(3*24*60*60)
label3 = label3.fillna(3*24*60*60)
label4 = label4.fillna(3*24*60*60)

In [75]:
train1.drop(['created'],axis =1, inplace = True )
train2.drop(['created'],axis =1, inplace = True )
train3.drop(['created'],axis =1, inplace = True )
train4.drop(['created'],axis =1, inplace = True )
ventana_ultima.drop(['created'],axis =1, inplace = True )


In [76]:
train1.shape,label1.shape

((160861, 29), (160861, 1))

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(train1, label1, test_size=0.2, random_state=123)

In [78]:
pa_reg = PassiveAggressiveRegressor()
#pa_reg.fit(X_train,y_train)


In [80]:
preds = pa_reg.predict(ventana_ultima)


In [37]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 28742.845503


In [79]:
pa_reg.partial_fit(train1,label1)
#pa_reg.partial_fit(train2,label2)
#pa_reg.partial_fit(train3,label3)
pa_reg.partial_fit(train4,label4)

/home/barbara/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/barbara/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


PassiveAggressiveRegressor(C=1.0, average=False, early_stopping=False,
                           epsilon=0.1, fit_intercept=True,
                           loss='epsilon_insensitive', max_iter=1000,
                           n_iter_no_change=5, random_state=None, shuffle=True,
                           tol=0.001, validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [61]:
preds = pa_reg.predict(ventana_ultima)


In [39]:
#########################################################
#ahora predigo para las ventanas...

In [81]:
preds-259200

array([10213.26158613, 14775.49988196, 15293.62531468, ...,
       16354.46536136,  7678.79344992, 17585.95861184])

In [ ]:
# LAS PREDICCIONES SE PASAN...